## 2012 play by play data

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sn
from sklearn.metrics import silhouette_samples, silhouette_score


sys.setrecursionlimit(100000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import 2012 nhl data set

In [22]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_pbp20122013.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_pbp20122013.csv')
da = da.drop('Unnamed: 0', axis=1)

- rename columns and create a season variable

In [23]:
da['Season'] = 2012
da = da.rename(columns={ 'Game_Id' : 'GameNumber', 'Ev_Zone': 'EventZone', 'Time_Elapsed': 'Time', 'Ev_Team': 'EventTeamCode', 'Description': 'EventDetail', 'Away_Team': 'ATeamCode', 'Home_Team':'HTeamCode', 'p1_name': 'EventPlayer1','p2_name': 'EventPlayer2', 'p3_name': 'EventPlayer3', 'awayPlayer1': 'APlayer1', 'awayPlayer2': 'APlayer2', 'awayPlayer3': 'APlayer3', 'awayPlayer4': 'APlayer4', 'awayPlayer5': 'APlayer5', 'awayPlayer6': 'APlayer6', 'homePlayer1': 'HPlayer1', 'homePlayer2': 'HPlayer2', 'homePlayer3': 'HPlayer3','homePlayer4': 'HPlayer4', 'homePlayer5': 'HPlayer5','homePlayer6': 'HPlayer6', 'Away_Score': 'AGoals', 'Home_Score': 'HGoals', 'Away_Goalie': 'AGoalie', 'Home_Goalie': 'HGoalie', 'Away_Players': 'PlayersA', 'Home_Players': 'PlayersH' })
da = da[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'Strength', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer1', 'APlayer2', 'APlayer3', 'APlayer4', 'APlayer5', 'APlayer6', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer1', 'HPlayer2', 'HPlayer3', 'HPlayer4', 'HPlayer5', 'HPlayer6', 'HGoalie', 'PlayersH']]
da = da.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- keep only regular season games and exclude irrelevant on-ice events

In [24]:
da = da[da['GameNumber'] <= 20720]
da = da[da['Period'] <= 4]
da = da[da['Period'] >= 1]
da = da[da['Event']!='STOP']
da = da[da['Event']!='EISTR']
da = da[da['Event']!='EIEND']
da = da[da['Event']!='PSTR']
da = da[da['Event']!='PEND']
da = da[da['Event']!='SOC']
da = da[da['Event']!='GEND']

- create an event number variable that will count the number of events per game. 
- create an advantage type variable for even strength, power play and short handed situations.

In [25]:
da['EventNumber'] = da.groupby(['Season', 'GameNumber']).cumcount()+1

In [26]:
da['AdvantageType'] = da.apply(lambda x: 'EV' if ((x['PlayersA'] == x['PlayersH']) & (x['PlayersA'] != 1) & (x['PlayersH'] != 1)) else 'PP' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'PP' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'SH' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'SH' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'PP' if ((x['PlayersA'] == 1 ) & (x['PlayersH'] == 1)) else np.nan, axis=1) 

In [27]:
da['AdvantageType'].value_counts()

EV    153540
PP     19994
SH      9499
Name: AdvantageType, dtype: int64

- reshape data from wide to long for away team players and home team players

In [28]:
db = da.copy()
a = [col for col in db.columns if 'APlayer' in col]
b = [col for col in db.columns if 'HPlayer' in col]
db = pd.lreshape(db, {'APlayer' : a, 'HPlayer' : b })
db.head()

,AGoalie,ATeamCode,AdvantageType,Date,Event,EventDetail,EventNumber,EventPlayer1,EventPlayer2,EventPlayer3,EventTeamCode,GameNumber,HGoalie,HTeamCode,Period,PlayersA,PlayersH,Season,Strength,Time,APlayer,HPlayer
0,MARC-ANDRE FLEURY,PIT,EV,2013-01-19,FAC,PIT won Neu. Zone - PIT #87 CROSBY vs PHI #28 ...,1,SIDNEY CROSBY,CLAUDE GIROUX,NaN,PIT,20001,ILYA BRYZGALOV,PHI,1,6,6,2012,5x5,0:00,SIDNEY CROSBY,BRAYDEN SCHENN
1,MARC-ANDRE FLEURY,PIT,EV,2013-01-19,HIT,"PHI #22 SCHENN HIT PIT #14 KUNITZ, Neu. Zone",2,LUKE SCHENN,CHRIS KUNITZ,NaN,PHI,20001,ILYA BRYZGALOV,PHI,1,6,6,2012,5x5,0:12,SIDNEY CROSBY,BRAYDEN SCHENN
2,MARC-ANDRE FLEURY,PIT,EV,2013-01-19,SHOT,"PIT ONGOAL - #44 ORPIK, Snap, Off. Zone, 52 ft.",3,BROOKS ORPIK,NaN,NaN,PIT,20001,ILYA BRYZGALOV,PHI,1,6,6,2012,5x5,0:29,SIDNEY CROSBY,BRAYDEN SCHENN
3,MARC-ANDRE FLEURY,PIT,EV,2013-01-19,FAC,PHI won Def. Zone - PIT #71 MALKIN vs PHI #14 ...,4,SEAN COUTURIER,EVGENI MALKIN,NaN,PHI,20001,ILYA BRYZGALOV,PHI,1,6,6,2012,5x5,0:29,ERIC TANGRADI,SEAN COUTURIER
4,MARC-ANDRE FLEURY,PIT,EV,2013-01-19,SHOT,"PHI ONGOAL - #25 TALBOT, Wrist, Off. Zone, 5 ft.",5,MAX TALBOT,NaN,NaN,PHI,20001,ILYA BRYZGALOV,PHI,1,6,6,2012,5x5,0:46,ERIC TANGRADI,SEAN COUTURIER


In [29]:
db = db[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'AdvantageType', 'Strength', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer', 'HGoalie', 'PlayersH']]
db = db.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- reshape data from wide to long for away and home team code and player respectfully.

In [30]:
dc = db.copy()
dc = dc.rename(columns={'EventTeamCode': 'EventTeam', 'EventPlayer1': 'EventP1', 'EventPlayer2': 'EventP2', 'EventPlayer3': 'EventP3', 'PlayersA':'PA', 'PlayersH': 'PH'})

In [31]:
a = [col for col in dc.columns if 'Code' in col]
b = [col for col in dc.columns if 'Player' in col]
c = [col for col in dc.columns if 'Goalie' in col]
d = [col for col in dc.columns if 'Position' in col]
dc = pd.lreshape(dc, {'TeamCode' : a, 'PlayerName' : b, 'Goalie': c})

In [32]:
dc = dc.rename(columns={'EventTeam': 'EventTeamCode', 'EventP1': 'EventPlayer1', 'EventP2': 'EventPlayer2', 'EventP3': 'EventPlayer3', 'PA':'PlayersA', 'PH': 'PlayersH'})

In [33]:
dc.shape

(2105393, 19)

### games played

- calculate the games each player participated in for the duration of the 2011 regular season.

In [34]:
s = dc.copy()
s = s[['Season', 'GameNumber', 'TeamCode', 'PlayerName']]
s = s.sort_values(['Season', 'GameNumber', 'TeamCode'], ascending=[True, True, True])
s = s.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [35]:
s['GP'] = s.groupby(['Season', 'TeamCode','PlayerName'])['GameNumber'].transform('count')
s = s.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
s = s[['Season', 'TeamCode', 'PlayerName', 'GP']]
s.head()

,Season,TeamCode,PlayerName,GP
1051674,2012,PHI,BRAYDEN SCHENN,47
1051675,2012,PHI,CLAUDE GIROUX,48
1051676,2012,PHI,SCOTT HARTNELL,32
1051677,2012,PHI,LUKE SCHENN,47
1051678,2012,PHI,KIMMO TIMONEN,45


In [38]:
s.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
dtype: int64

In [39]:
s.shape

(979, 4)

### shots for and against

- calculate the shots against each player faced througout the duration of a season. 
- create shot for and shot against variables. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [326]:
sa = dc.copy()
sa = sa[sa['Event'].isin(['SHOT','GOAL'])]

In [327]:
sa['ShotF'] = sa.apply(lambda x: 1 if x['EventTeamCode'] == x['TeamCode'] else 0, axis=1)
sa['ShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotF'].transform('sum')

sa['ShotA'] = sa.apply(lambda x: 1 if x['EventTeamCode'] != x['TeamCode'] else 0, axis=1)
sa['ShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotA'].transform('sum')

- create shot for and shot against variables for even strength situations only. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [328]:
sa['EVShotF'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
sa['EVShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotF'].transform('sum')

sa['EVShotA'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)
sa['EVShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotA'].transform('sum')

- calculate the shot differential for all shots and even strength shots.

In [329]:
sa['DShots'] = sa['ShotsF'] - sa['ShotsA']
sa['EVDShots'] = sa['EVShotsF'] - sa['EVShotsA']

In [330]:
sa = sa [['Season', 'TeamCode', 'PlayerName', 'ShotsF', 'ShotsA', 'DShots', 'EVShotsF', 'EVShotsA', 'EVDShots' ]]
sa = sa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
sa.head()

,Season,TeamCode,PlayerName,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots
12,2012,PIT,SIDNEY CROSBY,502,327,175,357,301,56
14,2012,PIT,PASCAL DUPUIS,439,442,-3,400,374,26
16,2012,PIT,CHRIS KUNITZ,569,355,214,385,353,32
18,2012,PIT,PAUL MARTIN,417,397,20,316,309,7
20,2012,PIT,BROOKS ORPIK,394,593,-199,390,465,-75


In [47]:
sa.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
dtype: int64

In [48]:
sa.shape

(979, 9)

### goals for and against

- create goals for and goals against variable. If event team code is the same as team code assign a value of 1 for goals for and a value of 0 for goals against. If event team code is different to team code, assign a value of 0 for golas for and a value of 1 for goals against. Display total goals for and against per player for a season.

In [49]:
gfa = dc.copy()
gfa = gfa[gfa['Event'] == 'GOAL']

In [50]:
gfa['EVGoalF'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
gfa['EVGoalA'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)

In [51]:
gfa['GoalF'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] == x['TeamCode']) else 0, axis=1)
gfa['GoalA'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] != x['TeamCode']) else 0, axis=1)

In [52]:
gfa['EVGoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalF'].transform('sum')
gfa['EVGoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalA'].transform('sum')
gfa['GoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalF'].transform('sum')
gfa['GoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalA'].transform('sum')

In [53]:
gfa = gfa[['Season', 'GameNumber', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA']]
gfa = gfa.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
gfa['Plus/Minus'] = gfa['EVGoalsF'] - gfa['EVGoalsA']
gfa = gfa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
gfa = gfa[['Season', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA', 'Plus/Minus']]
gfa.head()

,Season,TeamCode,PlayerName,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus
433,2012,PIT,BRANDON SUTTER,31,37,25,22,3
435,2012,PIT,TYLER KENNEDY,22,19,15,19,-4
437,2012,PIT,PASCAL DUPUIS,60,31,55,22,33
439,2012,PIT,MATT NISKANEN,33,27,29,22,7
441,2012,PIT,PAUL MARTIN,54,33,37,23,14


In [56]:
gfa.shape

(948, 8)

### create dataframes for all on-ice event types

#### goals dataframe (dg)

In [57]:
dg = dc.copy()
dg = dg[dg['Event'] == 'GOAL']

In [58]:
dg = dg[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dg = dg.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a goal variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [59]:
dg['Goal'] = dg.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dg = dg.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dg['Goals'] = dg.groupby(['Season','TeamCode', 'PlayerName'])['Goal'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [60]:
dg = dg.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dg = dg.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dg = dg[['Season', 'TeamCode', 'PlayerName', 'Goals']]
dg.head()

,Season,TeamCode,PlayerName,Goals
433,2012,PIT,PAUL MARTIN,6
509,2012,PIT,JAMES NEAL,21
617,2012,PHI,CLAUDE GIROUX,13
1502,2012,PIT,CHRIS KUNITZ,22
1889,2012,OTT,MILAN MICHALEK,4


In [63]:
dg.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Goals         0
dtype: int64

In [64]:
dg.shape

(652, 4)

#### assists dataframe (dast)

In [65]:
dast = dc.copy()
dast = dast[dast['Event'] == 'GOAL']

In [66]:
dast = dast[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer2', 'EventPlayer3']]
dast = dast.rename(columns={'EventPlayer2': 'Assist1Name', 'EventPlayer3': 'Assist2Name', 'EventTeamCode':'TeamCode'})

- reshape assist players 1 and 2 to create one column of assist players

In [67]:
a = [col for col in dast.columns if 'Name' in col]
dast = pd.lreshape(dast, {'PlayerName' : a})

- create an assist variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [68]:
dast['Assist'] = dast.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dast = dast.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dast['Assists'] = dast.groupby(['Season','TeamCode', 'PlayerName'])['Assist'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [69]:
dast = dast.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dast = dast.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dast = dast[['Season', 'TeamCode', 'PlayerName', 'Assists']]
dast.head()

,Season,TeamCode,PlayerName,Assists
0,2012,PIT,MATT NISKANEN,10
5,2012,PIT,EVGENI MALKIN,24
11,2012,PHI,SCOTT HARTNELL,3
17,2012,PIT,PAUL MARTIN,17
22,2012,OTT,PATRICK WIERCIOCH,14


In [71]:
dast.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Assists       0
dtype: int64

In [72]:
dast.shape

(758, 4)

#### shot dataframe (ds)

In [73]:
ds = dc.copy()
ds = ds[ds['Event'] == 'SHOT']

In [77]:
ds = ds[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
ds = ds.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})
ds.head()

,Season,GameNumber,EventNumber,Event,TeamCode,PlayerName
12,2012,20001,3,SHOT,PIT,BROOKS ORPIK
14,2012,20001,3,SHOT,PIT,BROOKS ORPIK
16,2012,20001,3,SHOT,PIT,BROOKS ORPIK
18,2012,20001,3,SHOT,PIT,BROOKS ORPIK
20,2012,20001,3,SHOT,PIT,BROOKS ORPIK


- create a shot variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [78]:
ds['Shot'] = ds.apply(lambda x: 1 if (x['Event'] == 'SHOT') else 0, axis=1)
ds = ds.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
ds['Shots'] = ds.groupby(['Season','TeamCode', 'PlayerName'])['Shot'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [79]:
ds = ds.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
ds = ds.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
ds = ds[['Season', 'TeamCode', 'PlayerName', 'Shots']]
ds.head()

,Season,TeamCode,PlayerName,Shots
12,2012,PIT,BROOKS ORPIK,32
24,2012,PHI,MAX TALBOT,36
88,2012,PHI,JAKUB VORACEK,107
122,2012,PIT,SIDNEY CROSBY,109
152,2012,PIT,MATT NISKANEN,63


In [81]:
ds.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Shots         0
dtype: int64

In [82]:
ds.shape

(856, 4)

#### block dataframe (db)

In [147]:
dbl = dc.copy()
dbl = dbl[dbl['Event'] == 'BLOCK']

In [148]:
dbl.shape

(236001, 19)

In [149]:
dbl['EventPlayer2'].isnull().sum()

0

In [150]:
dbl['EventPlayer3'].isnull().sum()

236001

In [151]:
dbl['BlockTeam'] = dbl.EventDetail.str.split(' ', expand = True)[6]
dbl['BlockTeamCode'] = dbl.BlockTeam.str.split(' ', expand = True)[0]

In [152]:
dbl = dbl[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventPlayer1', 'BlockTeamCode']]
dbl = dbl.rename(columns={'EventPlayer1': 'PlayerName', 'BlockTeamCode':'TeamCode'})

- create a block variable. If event type is a block assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [153]:
dbl['Block'] = dbl.apply(lambda x: 1 if (x['Event'] == 'BLOCK') else 0, axis=1)
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dbl['Blocks'] = dbl.groupby(['Season','TeamCode', 'PlayerName'])['Block'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [154]:
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dbl = dbl.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dbl = dbl[['Season', 'TeamCode', 'PlayerName', 'Blocks']]
dbl.head()

,Season,TeamCode,PlayerName,Blocks
42,2012,PIT,PAUL MARTIN,50
128,2012,PHI,JAKUB VORACEK,12
189,2012,PIT,BROOKS ORPIK,113
398,2012,PHI,KIMMO TIMONEN,76
423,2012,PHI,ERIC WELLWOOD,1


In [155]:
dbl.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Blocks        0
dtype: int64

In [156]:
dbl.shape

(952, 4)

#### hit dataframe (dh)

In [157]:
dhit = dc.copy()
dhit = dhit[dhit['Event'] == 'HIT']

In [158]:
dhit.shape

(404349, 19)

In [159]:
dhit['EventPlayer2'].isnull().sum()

0

In [160]:
dhit['EventPlayer3'].isnull().sum()

404349

In [161]:
dhit = dhit[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dhit = dhit.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a hit variable. If event type is a hit assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [162]:
dhit['Hit'] = dhit.apply(lambda x: 1 if (x['Event'] == 'HIT') else 0, axis=1)
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dhit['Hits'] = dhit.groupby(['Season','TeamCode', 'PlayerName'])['Hit'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [163]:
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dhit = dhit.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dhit = dhit[['Season', 'TeamCode', 'PlayerName', 'Hits']]
dhit.head()

,Season,TeamCode,PlayerName,Hits
6,2012,PHI,LUKE SCHENN,187
30,2012,PIT,ERIC TANGRADI,6
54,2012,PIT,CHRIS KUNITZ,104
66,2012,PIT,TANNER GLASS,124
104,2012,PIT,EVGENI MALKIN,12


In [164]:
dhit.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Hits          0
dtype: int64

In [165]:
dhit.shape

(878, 4)

#### penalty dataframe (dpen)

In [301]:
dp = dc.copy()
dp = dp[dp['Event'] == 'PENL']

- use event detail to find the duration of a given penalty and to assign it to the proper player. Major penalty is 5 minutes, so a value of 5 is assigned for every event that had a major penalty.

In [302]:
dp['Pen'] = dp.EventDetail.str.split(')', expand = True)[0]
dp['Penal'] = dp.Pen.str.split('(', expand = True)[1]
dp['Penalt'] = dp.Penal.str.split(' ', expand = True)[0]
dp['Penalty'] = dp.apply(lambda x: 5 if x['Penalt'] == 'maj' else x['Penalt'], axis=1)
dp['Penalty'] = dp['Penalty'].convert_objects(convert_numeric=True)
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


- use event detail to find penalty team code and penalty player name.

In [304]:
dp['PenaltyTeamCode'] = dp.EventDetail.str.split('\\s', expand = True)[0]
dp['PN'] = dp.EventDetail.str.split('\\s', expand = True)[1]
dp['PNumber'] = dp.PN.str.split('#', expand = True)[1]
dp['PenaltyPlayerNumber'] = dp.PNumber.str.split(' ', expand = True)[0]
dp['PenaltyPlayerLName'] = dp.EventDetail.str.split('\\s', expand = True)[2]
#dp['PenaltyPlayerLName'] = dp.PPlLN.str.split(' ', expand = True)[0]
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName']]

- seperate player name into first and last. The purpose is to connect the correct player name to penalty player name.

In [305]:
dp['PlayerFName'] = dp.PlayerName.str.split('\\s', expand = True)[0]
dp['PlayerLName'] = dp.PlayerName.str.split('\\s', expand = True)[1]
dp['PenaltyPlayerFName'] = dp.apply(lambda x: x['PlayerFName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName'])) else np.nan, axis=1)
dp['PenaltyPlayerFName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyPlayerFName'].apply(lambda x: x.ffill().bfill())

In [306]:
dp['PenaltyName'] = dp.apply(lambda x: x['PlayerName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName']) & (x['PlayerFName'] == x['PenaltyPlayerFName'])) else np.nan, axis=1)
dp['PenaltyName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerLName', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyName'].apply(lambda x: x.ffill().bfill())

- drop duplicates per game and event number. Calculate the total number of penalties for each player.

In [307]:
dp = dp.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])
dp['Penalties'] = dp.groupby(['Season', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])['Penalty'].transform('sum')
dp['Penalties'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PlayerName'])['Penalties'].apply(lambda x: x.ffill().bfill())
#dp.head()

- drop duplicates to keep one observation per player for the season.

In [308]:
dp = dp[['Season', 'PenaltyTeamCode', 'PenaltyName', 'Penalties']]
dp = dp.drop_duplicates(['Season', 'PenaltyTeamCode', 'PenaltyName'])
dp = dp.rename(columns={'PenaltyTeamCode':'TeamCode', 'PenaltyName':'PlayerName'})
dp = dp[['Season', 'TeamCode', 'PlayerName', 'Penalties']]
dp.head()

,Season,TeamCode,PlayerName,Penalties
72,2012,PIT,SIMON DESPRES,20.0
212,2012,PIT,BRANDON SUTTER,4.0
372,2012,PHI,BRAYDON COBURN,41.0
450,2012,PIT,DUSTIN JEFFREY,2.0
1068,2012,PHI,NICKLAS GROSSMANN,21.0


- keep observations that are not null. Since a team can be allocated a penalty, those observations are excluded from the data.

In [310]:
dp['PlayerName'].isnull().sum()

30

In [311]:
dp = dp[dp['PlayerName'].notnull()]

In [312]:
dp.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Penalties     0
dtype: int64

In [313]:
dp.shape

(789, 4)

####  miss dataframe (dmiss)

In [177]:
dmiss = dc.copy()
dmiss = dmiss[dmiss['Event'] == 'MISS']

In [178]:
dmiss.shape

(188209, 19)

In [179]:
dmiss['EventPlayer2'].isnull().sum()

188209

In [180]:
dmiss['EventPlayer3'].isnull().sum()

188209

In [181]:
dmiss = dmiss[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dmiss = dmiss.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a miss variable. If event type is a miss assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [182]:
dmiss['Miss'] = dmiss.apply(lambda x: 1 if (x['Event'] == 'MISS') else 0, axis=1)
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dmiss['Misses'] = dmiss.groupby(['Season','TeamCode', 'PlayerName'])['Miss'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [183]:
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dmiss = dmiss.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dmiss = dmiss[['Season', 'TeamCode', 'PlayerName', 'Misses']]
dmiss.head()

,Season,TeamCode,PlayerName,Misses
93,2012,PHI,ANDREJ MESZAROS,6
228,2012,PHI,JAKUB VORACEK,36
253,2012,PHI,SEAN COUTURIER,22
306,2012,PHI,BRAYDON COBURN,29
388,2012,PIT,SIDNEY CROSBY,44


In [184]:
dmiss.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Misses        0
dtype: int64

In [185]:
dmiss.shape

(842, 4)

####  takeaway dataframe (dtake)

In [186]:
dtake = dc.copy()
dtake = dtake[dtake['Event'] == 'TAKE']

In [187]:
dtake.shape

(107567, 19)

In [188]:
dtake['EventPlayer2'].isnull().sum()

107567

In [189]:
dtake['EventPlayer3'].isnull().sum()

107567

In [190]:
dtake = dtake[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dtake = dtake.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a takeaway variable. If event type is a takeaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [191]:
dtake['Takeaway'] = dtake.apply(lambda x: 1 if (x['Event'] == 'TAKE') else 0, axis=1)
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dtake['Takeaways'] = dtake.groupby(['Season','TeamCode', 'PlayerName'])['Takeaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [192]:
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dtake = dtake.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dtake = dtake[['Season', 'TeamCode', 'PlayerName', 'Takeaways']]
dtake.head()

,Season,TeamCode,PlayerName,Takeaways
98,2012,PIT,EVGENI MALKIN,23
116,2012,PIT,JAMES NEAL,11
146,2012,PIT,SIDNEY CROSBY,15
486,2012,PHI,SCOTT HARTNELL,11
599,2012,PIT,MATT NISKANEN,15


In [193]:
dtake.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Takeaways     0
dtype: int64

In [194]:
dtake.shape

(793, 4)

####  giveaway dataframe (dgive)

In [195]:
dgive = dc.copy()
dgive = dgive[dgive['Event'] == 'GIVE']

In [196]:
dgive.shape

(128664, 19)

In [197]:
dgive['EventPlayer2'].isnull().sum()

128664

In [198]:
dgive['EventPlayer3'].isnull().sum()

128664

In [199]:
dgive = dgive[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dgive = dgive.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a giveaway variable. If event type is a giveaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [200]:
dgive['Giveaway'] = dgive.apply(lambda x: 1 if (x['Event'] == 'GIVE') else 0, axis=1)
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dgive['Giveaways'] = dgive.groupby(['Season','TeamCode', 'PlayerName'])['Giveaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [201]:
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dgive = dgive.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dgive = dgive[['Season', 'TeamCode', 'PlayerName', 'Giveaways']]
dgive.head()

,Season,TeamCode,PlayerName,Giveaways
36,2012,PHI,BRAYDON COBURN,31
164,2012,PHI,JAKUB VORACEK,25
276,2012,PIT,JAMES NEAL,7
383,2012,PHI,NICKLAS GROSSMANN,17
403,2012,PHI,LUKE SCHENN,31


In [202]:
dgive.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Giveaways     0
dtype: int64

In [203]:
dgive.shape

(890, 4)

### import 2012 player shifts

In [227]:
dshift = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_shifts20122013.csv')
#dshift = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_shifts20122013.csv')
dshift = dshift.drop('Unnamed: 0', axis=1)

- rename variables, keep only regular season games and create a season variable.

In [228]:
dshift['Season'] = 2012
dshift= dshift.rename(columns={ 'Game_Id' : 'GameNumber', 'Team': 'TeamCode', 'Player': 'PlayerName' })
dshift = dshift[dshift['GameNumber'] <= 20720]
dshift = dshift[['Season', 'Date', 'GameNumber', 'Period', 'TeamCode', 'PlayerName', 'Player_Id', 'Start', 'End', 'Duration']]
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration
0,2012,2013-01-19,20003,1,L.A,DREW DOUGHTY,8474563,0.0,35.0,35.0
1,2012,2013-01-19,20003,1,L.A,ROB SCUDERI,8467452,0.0,35.0,35.0
2,2012,2013-01-19,20003,1,CHI,BRENT SEABROOK,8470607,0.0,44.0,44.0
3,2012,2013-01-19,20003,1,CHI,DUNCAN KEITH,8470281,0.0,44.0,44.0
4,2012,2013-01-19,20003,1,L.A,DWIGHT KING,8474100,0.0,44.0,44.0


- calcuate the seconds an individual played per game. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [229]:
dshift['Seconds'] = dshift.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])['Duration'].transform('sum')
dshift['Minutes'] = dshift['Seconds']/60
dshift['Minutes'] = dshift['Minutes'].round(2)

In [230]:
dshift = dshift.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [231]:
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes
0,2012,2013-01-19,20003,1,L.A,DREW DOUGHTY,8474563,0.0,35.0,35.0,1548.0,25.80
1,2012,2013-01-19,20003,1,L.A,ROB SCUDERI,8467452,0.0,35.0,35.0,1403.0,23.38
2,2012,2013-01-19,20003,1,CHI,BRENT SEABROOK,8470607,0.0,44.0,44.0,1381.0,23.02
3,2012,2013-01-19,20003,1,CHI,DUNCAN KEITH,8470281,0.0,44.0,44.0,1364.0,22.73
4,2012,2013-01-19,20003,1,L.A,DWIGHT KING,8474100,0.0,44.0,44.0,705.0,11.75


- calcuate the seconds an individual played for the whole regular season. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [232]:
dshift['TSeconds'] = dshift.groupby(['Season', 'TeamCode', 'PlayerName'])['Seconds'].transform('sum')
dshift['TOI'] = dshift['TSeconds']/60
dshift['TOI'] = dshift['TOI'].round(0)
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes,TSeconds,TOI
0,2012,2013-01-19,20003,1,L.A,DREW DOUGHTY,8474563,0.0,35.0,35.0,1548.0,25.80,76016.0,1267.0
1,2012,2013-01-19,20003,1,L.A,ROB SCUDERI,8467452,0.0,35.0,35.0,1403.0,23.38,62759.0,1046.0
2,2012,2013-01-19,20003,1,CHI,BRENT SEABROOK,8470607,0.0,44.0,44.0,1381.0,23.02,62037.0,1034.0
3,2012,2013-01-19,20003,1,CHI,DUNCAN KEITH,8470281,0.0,44.0,44.0,1364.0,22.73,67986.0,1133.0
4,2012,2013-01-19,20003,1,L.A,DWIGHT KING,8474100,0.0,44.0,44.0,705.0,11.75,35972.0,600.0


- keep only observation per player for the season

In [233]:
dshift = dshift.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dshift = dshift[['Season', 'TeamCode', 'PlayerName', 'TOI']]
dshift.head()

,Season,TeamCode,PlayerName,TOI
0,2012,L.A,DREW DOUGHTY,1267.0
1,2012,L.A,ROB SCUDERI,1046.0
2,2012,CHI,BRENT SEABROOK,1034.0
3,2012,CHI,DUNCAN KEITH,1133.0
4,2012,L.A,DWIGHT KING,600.0


In [234]:
dshift.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
TOI           0
dtype: int64

In [235]:
dshift.shape

(979, 4)

### import 2012 player positions

In [263]:
dpos = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_positions20122013.csv')
#dpos = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_positions20122013.csv')
dpos = dpos.drop('Unnamed: 7', axis=1)

In [264]:
dpos.head()

,LastName,FirstName,TeamCode,Position,LName,FName,PlayerName
0,Abdelkader,Justin,DET,LW/C,ABDELKADER,JUSTIN,JUSTIN ABDELKADER
1,Adam,Luke,BUF,C/LW,ADAM,LUKE,LUKE ADAM
2,Adams,Craig,PIT,RW,ADAMS,CRAIG,CRAIG ADAMS
3,Akeson,Jason,PHI,RW/C,AKESON,JASON,JASON AKESON
4,Alberts,Andrew,VAN,D,ALBERTS,ANDREW,ANDREW ALBERTS


In [265]:
dpos['Season'] = 2012
dpos= dpos.rename(columns={ 'Team': 'TeamCode', 'Pos ': 'Position'})
dpos = dpos[['Season', 'TeamCode', 'PlayerName', 'Position']]
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2012,DET,JUSTIN ABDELKADER,LW/C
1,2012,BUF,LUKE ADAM,C/LW
2,2012,PIT,CRAIG ADAMS,RW
3,2012,PHI,JASON AKESON,RW/C
4,2012,VAN,ANDREW ALBERTS,D


In [266]:
dpos.dtypes

Season         int64
TeamCode      object
PlayerName    object
Position      object
dtype: object

In [267]:
dpos['Position'] = dpos.apply(lambda x: 'C' if x['Position'] == 'C/LW' else 'C' if x['Position'] == 'C/RW' else 'C' if x['Position'] == 'C/RW/LW' else 'C' if x['Position'] == 'C/LW/RW' else 'W' if x['Position'] == 'RW' else 'W' if x['Position'] == 'LW' else 'W' if x['Position'] == 'LW/C' else 'W' if x['Position'] == 'RW/C' else 'W' if x['Position'] == 'RW/LW' else 'W' if x['Position'] == 'LW/RW' else 'W' if x['Position'] == 'RW/LW/C' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'LW/D' else 'W' if x['Position'] == 'LW/C/RW' else 'W' if x['Position'] == 'LW/RW/C' else 'D' if x['Position'] == 'D/LW' else 'D' if x['Position'] == 'D/RW' else 'C' if x['Position'] == 'C/D' else 'W' if x['Position'] == 'RW/D' else x['Position'] , axis=1)
dpos['Position'].value_counts()

W    315
D    290
C    234
G     82
Name: Position, dtype: int64

In [268]:
dpos['TeamCode'] = dpos.apply(lambda x: 'L.A' if x['TeamCode'] == 'LAK' else 'N.J' if x['TeamCode'] == 'NJD' else 'S.J' if x['TeamCode'] == 'SJS' else 'T.B' if x['TeamCode'] == 'TBL' else x['TeamCode'] , axis=1)
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2012,DET,JUSTIN ABDELKADER,W
1,2012,BUF,LUKE ADAM,C
2,2012,PIT,CRAIG ADAMS,W
3,2012,PHI,JASON AKESON,W
4,2012,VAN,ANDREW ALBERTS,D


In [269]:
dpos.shape

(921, 4)

### merge data frames

- merge games played, goals and assists.

In [314]:
dd = pd.merge(s, dpos, on=['Season', 'TeamCode', 'PlayerName'], how='left').merge(dg,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dast,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')

- forward and backward fill players that have been traded during the season. Fill in positions for three players.

In [315]:
dd['Position'] = dd.groupby(['PlayerName'])['Position'].apply(lambda x: x.ffill().bfill())
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists
0,2012,PHI,BRAYDEN SCHENN,47,C,8.0,18.0
1,2012,PHI,CLAUDE GIROUX,48,W,13.0,35.0
2,2012,PHI,SCOTT HARTNELL,32,W,8.0,3.0
3,2012,PHI,LUKE SCHENN,47,D,3.0,8.0
4,2012,PHI,KIMMO TIMONEN,45,D,5.0,24.0


In [316]:
dd.isnull().sum()

Season          0
TeamCode        0
PlayerName      0
GP              0
Position        0
Goals         327
Assists       221
dtype: int64

- fill in goals and assists with 0 for players who had no points.

In [317]:
dd['Goals'] = dd['Goals'].fillna(0)
dd['Assists'] = dd['Assists'].fillna(0)
dd['Points'] = dd['Goals'] + dd['Assists']
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points
0,2012,PHI,BRAYDEN SCHENN,47,C,8.0,18.0,26.0
1,2012,PHI,CLAUDE GIROUX,48,W,13.0,35.0,48.0
2,2012,PHI,SCOTT HARTNELL,32,W,8.0,3.0,11.0
3,2012,PHI,LUKE SCHENN,47,D,3.0,8.0,11.0
4,2012,PHI,KIMMO TIMONEN,45,D,5.0,24.0,29.0


In [318]:
dd.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
dtype: int64

In [319]:
de = pd.merge(dd, gfa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(ds,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(sa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dbl,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dhit,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dp,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dtake,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dgive,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dmiss,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dshift,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')
de.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus,Shots,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots,Blocks,Hits,Penalties,Takeaways,Giveaways,Misses,TOI
0,2012,PHI,BRAYDEN SCHENN,47,C,8.0,18.0,26.0,38.0,30.0,25.0,30.0,-5.0,71.0,372,310,62,287,310,-23,22.0,109.0,22.0,19.0,11.0,31.0,730.0
1,2012,PHI,CLAUDE GIROUX,48,W,13.0,35.0,48.0,66.0,44.0,37.0,39.0,-2.0,124.0,594,459,135,405,404,1,29.0,57.0,22.0,17.0,30.0,57.0,1015.0
2,2012,PHI,SCOTT HARTNELL,32,W,8.0,3.0,11.0,25.0,13.0,11.0,13.0,-2.0,66.0,298,190,108,201,190,11,21.0,68.0,70.0,11.0,21.0,35.0,507.0
3,2012,PHI,LUKE SCHENN,47,D,3.0,8.0,11.0,42.0,46.0,41.0,37.0,4.0,78.0,470,476,-6,460,390,70,100.0,187.0,34.0,14.0,31.0,23.0,1028.0
4,2012,PHI,KIMMO TIMONEN,45,D,5.0,24.0,29.0,58.0,36.0,32.0,25.0,7.0,73.0,539,412,127,371,318,53,76.0,31.0,36.0,13.0,21.0,35.0,979.0


- fill in with 0 for all nan values since not all players have been on the ice for a goal for or a goal against. 

In [320]:
de['GoalsF'] = de['GoalsF'].fillna(0)
de['GoalsA'] = de['GoalsA'].fillna(0)
de['EVGoalsF'] = de['EVGoalsF'].fillna(0)
de['EVGoalsA'] = de['EVGoalsA'].fillna(0)
de['Plus/Minus'] = de['Plus/Minus'].fillna(0)
de['Shots'] = de['Shots'].fillna(0)
de['Hits'] = de['Hits'].fillna(0)
de['Blocks'] = de['Blocks'].fillna(0)
de['Penalties'] = de['Penalties'].fillna(0)
de['Giveaways'] = de['Giveaways'].fillna(0)
de['Takeaways'] = de['Takeaways'].fillna(0)
de['Misses'] = de['Misses'].fillna(0)

In [321]:
de.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
GoalsF        0
GoalsA        0
EVGoalsF      0
EVGoalsA      0
Plus/Minus    0
Shots         0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
Blocks        0
Hits          0
Penalties     0
Takeaways     0
Giveaways     0
Misses        0
TOI           0
dtype: int64

### total player stats for the season

- players that have been traded need to be evaluated for their season performance. Group by sesaon and player name to calcute the sum of each stat. 

In [322]:
de['TGP'] = de.groupby(['Season', 'PlayerName'])['GP'].transform('sum')
de['TGoals'] = de.groupby(['Season', 'PlayerName'])['Goals'].transform('sum')
de['TAssists'] = de.groupby(['Season', 'PlayerName'])['Assists'].transform('sum')
de['TPoints'] = de.groupby(['Season', 'PlayerName'])['Points'].transform('sum')
de['TGoalsF'] = de.groupby(['Season', 'PlayerName'])['GoalsF'].transform('sum')
de['TGoalsA'] = de.groupby(['Season', 'PlayerName'])['GoalsA'].transform('sum')
de['TEVGoalsF'] = de.groupby(['Season', 'PlayerName'])['EVGoalsF'].transform('sum')
de['TEVGoalsA'] = de.groupby(['Season', 'PlayerName'])['EVGoalsA'].transform('sum')
de['TPlus/Minus'] = de.groupby(['Season', 'PlayerName'])['Plus/Minus'].transform('sum')
de['TShots'] = de.groupby(['Season', 'PlayerName'])['Shots'].transform('sum')
de['TShotsF'] = de.groupby(['Season', 'PlayerName'])['ShotsF'].transform('sum')
de['TShotsA'] = de.groupby(['Season', 'PlayerName'])['ShotsA'].transform('sum')
de['TDShots'] = de.groupby(['Season', 'PlayerName'])['DShots'].transform('sum')
de['TEVShotsF'] = de.groupby(['Season', 'PlayerName'])['EVShotsF'].transform('sum')
de['TEVShotsA'] = de.groupby(['Season', 'PlayerName'])['EVShotsA'].transform('sum')
de['TEVDShots'] = de.groupby(['Season', 'PlayerName'])['EVDShots'].transform('sum')
de['TBlocks'] = de.groupby(['Season', 'PlayerName'])['Blocks'].transform('sum')
de['THits'] = de.groupby(['Season', 'PlayerName'])['Hits'].transform('sum')
de['TPenalties'] = de.groupby(['Season', 'PlayerName'])['Penalties'].transform('sum')
de['TTakeaways'] = de.groupby(['Season', 'PlayerName'])['Takeaways'].transform('sum')
de['TGiveaways'] = de.groupby(['Season', 'PlayerName'])['Giveaways'].transform('sum')
de['TMisses'] = de.groupby(['Season', 'PlayerName'])['Misses'].transform('sum')
de['TTOI'] = de.groupby(['Season', 'PlayerName'])['TOI'].transform('sum')

- divide all stats with total time on ice to scale the data for cluster analysis

In [323]:
de['TOIGoals'] = de['TGoals'] / de['TTOI']
de['TOIAssists'] = de['TAssists'] / de['TTOI']
de['TOIPoints'] = de['TPoints'] / de['TTOI']
de['TOIGoalsF'] = de['TGoalsF'] / de['TTOI']
de['TOIGoalsA'] = de['TGoalsA'] / de['TTOI']
de['TOIEVGoalsF'] = de['TEVGoalsF'] / de['TTOI']
de['TOIEVGoalsA'] = de['TEVGoalsA'] / de['TTOI']
de['TOIPlus/Minus'] = de['TPlus/Minus'] / de['TTOI']
de['TOIShots'] = de['TShots'] / de['TTOI']
de['TOIShotsF'] = de['TShotsF'] / de['TTOI']
de['TOIShotsA'] = de['TShotsA'] / de['TTOI']
de['TOIDShots'] = de['TDShots'] / de['TTOI']
de['TOIEVShotsF'] = de['TEVShotsF'] / de['TTOI']
de['TOIEVShotsA'] = de['TEVShotsA'] / de['TTOI']
de['TOIEVDShots'] = de['TEVDShots'] / de['TTOI']
de['TOIBlocks'] = de['TBlocks'] / de['TTOI']
de['TOIHits'] = de['THits'] / de['TTOI']
de['TOIPenalties'] = de['TPenalties'] / de['TTOI']
de['TOITakeaways'] = de['TTakeaways'] / de['TTOI']
de['TOIGiveaways'] = de['TGiveaways'] / de['TTOI']
de['TOIMisses'] = de['TMisses'] / de['TTOI']

- keep season total variables and scaled variables

In [324]:
de = de[['Season', 'TeamCode', 'PlayerName', 'Position', 'TGP', 'TGoals', 'TAssists', 'TPoints', 'TGoalsF', 'TGoalsA', 'TEVGoalsF', 'TEVGoalsA', 'TPlus/Minus', 'TShots', 'TShotsF', 'TShotsA', 'TDShots', 'TEVShotsF', 'TEVShotsA', 'TEVDShots', 'TBlocks', 'THits', 'TPenalties', 'TTakeaways', 'TGiveaways', 'TMisses', 'TTOI', 'TOIGoals', 'TOIAssists', 'TOIPoints', 'TOIGoalsF', 'TOIGoalsA', 'TOIEVGoalsF', 'TOIEVGoalsA', 'TOIPlus/Minus', 'TOIShots', 'TOIShotsF', 'TOIShotsA', 'TOIDShots', 'TOIEVShotsF', 'TOIEVShotsA', 'TOIEVDShots', 'TOIBlocks', 'TOIHits', 'TOIPenalties', 'TOITakeaways', 'TOIGiveaways', 'TOIMisses']]
de.head()

,Season,TeamCode,PlayerName,Position,TGP,TGoals,TAssists,TPoints,TGoalsF,TGoalsA,TEVGoalsF,TEVGoalsA,TPlus/Minus,TShots,TShotsF,TShotsA,TDShots,TEVShotsF,TEVShotsA,TEVDShots,TBlocks,THits,TPenalties,TTakeaways,TGiveaways,TMisses,TTOI,TOIGoals,TOIAssists,TOIPoints,TOIGoalsF,TOIGoalsA,TOIEVGoalsF,TOIEVGoalsA,TOIPlus/Minus,TOIShots,TOIShotsF,TOIShotsA,TOIDShots,TOIEVShotsF,TOIEVShotsA,TOIEVDShots,TOIBlocks,TOIHits,TOIPenalties,TOITakeaways,TOIGiveaways,TOIMisses
0,2012,PHI,BRAYDEN SCHENN,C,47,8.0,18.0,26.0,38.0,30.0,25.0,30.0,-5.0,71.0,372,310,62,287,310,-23,22.0,109.0,22.0,19.0,11.0,31.0,730.0,0.010959,0.024658,0.035616,0.052055,0.041096,0.034247,0.041096,-0.006849,0.097260,0.509589,0.424658,0.084932,0.393151,0.424658,-0.031507,0.030137,0.149315,0.030137,0.026027,0.015068,0.042466
1,2012,PHI,CLAUDE GIROUX,W,48,13.0,35.0,48.0,66.0,44.0,37.0,39.0,-2.0,124.0,594,459,135,405,404,1,29.0,57.0,22.0,17.0,30.0,57.0,1015.0,0.012808,0.034483,0.047291,0.065025,0.043350,0.036453,0.038424,-0.001970,0.122167,0.585222,0.452217,0.133005,0.399015,0.398030,0.000985,0.028571,0.056158,0.021675,0.016749,0.029557,0.056158
2,2012,PHI,SCOTT HARTNELL,W,32,8.0,3.0,11.0,25.0,13.0,11.0,13.0,-2.0,66.0,298,190,108,201,190,11,21.0,68.0,70.0,11.0,21.0,35.0,507.0,0.015779,0.005917,0.021696,0.049310,0.025641,0.021696,0.025641,-0.003945,0.130178,0.587771,0.374753,0.213018,0.396450,0.374753,0.021696,0.041420,0.134122,0.138067,0.021696,0.041420,0.069034
3,2012,PHI,LUKE SCHENN,D,47,3.0,8.0,11.0,42.0,46.0,41.0,37.0,4.0,78.0,470,476,-6,460,390,70,100.0,187.0,34.0,14.0,31.0,23.0,1028.0,0.002918,0.007782,0.010700,0.040856,0.044747,0.039883,0.035992,0.003891,0.075875,0.457198,0.463035,-0.005837,0.447471,0.379377,0.068093,0.097276,0.181907,0.033074,0.013619,0.030156,0.022374
4,2012,PHI,KIMMO TIMONEN,D,45,5.0,24.0,29.0,58.0,36.0,32.0,25.0,7.0,73.0,539,412,127,371,318,53,76.0,31.0,36.0,13.0,21.0,35.0,979.0,0.005107,0.024515,0.029622,0.059244,0.036772,0.032686,0.025536,0.007150,0.074566,0.550562,0.420838,0.129724,0.378958,0.324821,0.054137,0.077630,0.031665,0.036772,0.013279,0.021450,0.035751


In [325]:
de.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/stats/2012_player_stats.csv', index='False', sep=',')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/stats/2011_player_stats.csv', index='False', sep=',')